### introduction to Bioinformatics
### homework 2
### Name : Amirmohammad Ezzati
### Student Id : 402212269

In [28]:
import numpy as np
import subprocess
import random

In [29]:

# FTP URL of the FASTQ file you want to download
ftp_url = 'ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR390/SRR390728/SRR390728_1.fastq.gz'

# Destination path where the downloaded file will be saved
destination_path = 'SRR390728_1.fastq.gz'

# Download data using wget
try:
    subprocess.run(['wget', ftp_url, '-O', destination_path], check=True)
    print(f"Downloaded data for SRR390728_1.fastq.gz")
except subprocess.CalledProcessError as e:
    print(f"Error: {e}")

Downloaded data for SRR390728_1.fastq.gz


In [30]:
#Now You have to read from SRR390728_1.fastq.gz file and save all of this file sequence in concatenated_sequence
#hint : use gzip library and read about fastq files and how they save genomes sequence


import gzip

file_ = gzip.open('SRR390728_1.fastq.gz', 'rt')
fastq = file_.read()

fastq_splitted = fastq.split('\n') # split fastq text line by line
first_idx = fastq_splitted.index('+') # each sequence appears just before '+' in above list

# so with bottom formula we can find all index of sequences in fastq_splitted list
indexOfSequences = [ 4*i + first_idx - 1 for i in range(1 + (len(fastq_splitted)-1 - first_idx+1)//4)] if first_idx != 0 else [ 4*i + first_idx - 1 for i in range(1 + (len(fastq_splitted)-1 - first_idx+1)//4)][1:]


concatenated_sequences = ''.join([fastq_splitted[i] for i in indexOfSequences])
concatenated_sequences

In [31]:
len(concatenated_sequences)

In [32]:
#because genome is too long we just get first 35000 of genomes sequence and work with this genome after it
#so only run this cell
refrence_genome = concatenated_sequences[0:35000]

### just run the cell to create short reads from genome

In [ ]:

# Minimum length of each short read
min_read_length = 90

# Function to generate random short reads
def generate_random_short_reads(sequence, min_length):
    short_reads = []
    remaining_sequence = sequence

    while len(remaining_sequence) >= min_length:
        read_length = random.randint(min(min_length, len(remaining_sequence)), min(len(remaining_sequence), 700))
        short_read = remaining_sequence[:read_length]
        short_reads.append(short_read)
        remaining_sequence = remaining_sequence[read_length:]

    return short_reads

def mutate(input):
    valid_inputs = ['A', 'C', 'T', 'G']
    num = random.randint(1, 1000)
    valid_inputs = [element for element in valid_inputs if element not in [input]]

    if num < 40 :
        shuffled_indices = list(range(len(valid_inputs)))
        random.shuffle(shuffled_indices)
        return valid_inputs[shuffled_indices[0]]
    return input


# Generate random short reads
copy_num = 4
short_reads = generate_random_short_reads(refrence_genome, min_read_length)

for i in range (copy_num):
    short_reads = short_reads + generate_random_short_reads(refrence_genome, min_read_length)


shuffled_indices = list(range(len(short_reads)))
random.shuffle(shuffled_indices)

# Create a new list to store shuffled short reads
short_reads = [short_reads[i] for i in shuffled_indices]

#mutations
for i in range (len(short_reads)):
    for j in range (len(short_reads[i])):
        mutate_char = mutate(short_reads[i][j])
        if mutate_char != short_reads[i][j]:
            if j != len(short_reads[i])-1:
                short_reads[i] = short_reads[i][:j]+mutate_char+short_reads[i][j+1:]
            else :
                short_reads[i] = short_reads[i][:j]+mutate_char


### just run the code below (it's just for calculating local alignment score with refrence genome)

In [ ]:

def smith_waterman(sequence1, sequence2, match=2, mismatch=-1, gap_open=-2, gap_extend=-1):
    len_seq1, len_seq2 = len(sequence1), len(sequence2)
    score_matrix = np.zeros((len_seq1 + 1, len_seq2 + 1), dtype=int)

    for i in range(1, len_seq1 + 1):
        for j in range(1, len_seq2 + 1):
            match_mismatch_score = match if sequence1[i - 1] == sequence2[j - 1] else mismatch
            diagonal_score = score_matrix[i - 1][j - 1] + match_mismatch_score
            gap_up_score = score_matrix[i - 1][j] + gap_extend if score_matrix[i - 1][j] > 0 else gap_open
            gap_left_score = score_matrix[i][j - 1] + gap_extend if score_matrix[i][j - 1] > 0 else gap_open

            score_matrix[i][j] = max(0, diagonal_score, gap_up_score, gap_left_score)

    # Find the maximum score in the score matrix
    max_score = np.max(score_matrix)

    return max_score


In [9]:
# Print 10 samples of short reads with length
#TODO

for i in range(10):
  print(short_reads[i])

AGTGCTGATGTGGGAGCTGTGCGTGCTGGCCGCGCCAGGGCATCAGTGCTGATGTGGGGCCACTTGAAACTGGAGTGAGAAGCTGGTTACTTTCTATTACATAATAGCCACTTGAGACTAGAGTGAGAAGGTGAGTAGCTGGTTACTTGCTTGCAGTGCATCAGAAGGCCCAGTGGCCTTGGCAGGGGCAGGGGTGGCCTCCAGGGCCAGTGGCCTTGGCAGAGGCAGGCGGGGCCTCGTGAATGAGGGCCAGTGGCCTTGGCAGAGGCAGGTGTTCCTTGCAGTGCATCAGAACCAAGGTTTCTGGGGTCCCCGCAGACGAAGTAGGGCCAGTGGCCTTGGCAGAATCTCTAGAACGTCCCCGCAG
TGTCGCTACAGAGTTACTCAGTTACTGCGTGGATCTCCTGTCGCAACTGTACAAGCGATGCACACTCCAAGTGCACGTTTGCCCGGCCCTGGGTACGCGTGAGTGGGTCAGAGTTACTGGAGCCCGGCCCTGGGTTGGCGTGAGTGGGTCAGGCTTAGAGTCCATGGAGCCCGGCCCTGGGTTGGCGTCACGTATTTAATACAGTATTGACTATTTGGATTTGCGCACATATTTAATACAGTATTGACTATTTGCATTTATCAACATTCTGAAAAAAGATCAACTGTTGAAGATCTAAAAAGATCAACCGTTGAACATCTTTAAGGTAAATTAAGATCAACTGTTGAAGATCTTTAAGGTATATTGCCATATTAGAAAAAC
GAGACACAGTCTCCAACGCCACCAAAGCCTCCGTCGCAGCTCGTACGTTGCTGATACAGCACAAGATGCCGGCCCAGACACAGTCTCCACCGCCACCAAAGCCGCTGCCAGCCAGACACAGTCTCCAACGCAACCAAAGCCTCCGAAATCATCCAGATTTTTCAC
TGTGATCCTGCAAAGCTCTAAGATACACATGGCCACTTTTGACATCACCCACATATGGGACTGGAGTTGGAAGGCCGGGCCTAA

In [41]:
print(len(short_reads))

453


In [45]:
def find_high_overlap(seq1, seq2):
  string1 = seq1
  string2= (len(seq1)-1)* " " + seq2

  highest_overlap=0

  for k in range (len(seq1)):
    overlap_num=0
    #check individual characters to see if they match
    for i in range(len(string1)):
      overlap_num=0
      if string1[i]==string2[i]:
        for w in range(i, len(string1)):
            if string1[w]==string2[w]:
              overlap_num+=1
            else:
                overlap_num=0
                break
      # if the match is in the middle of the sequences
      elif string1[i]!=" " and string2[i]!=" ":
          overlap_num=0
          break
      if overlap_num > highest_overlap:
        highest_overlap = overlap_num
    if overlap_num > highest_overlap:
      highest_overlap = overlap_num
    string2= string2[1:]+" "

  return (highest_overlap)


def merge(seq1, seq2, found_overlap):
  string1 = seq1
  string2= (len(seq1)-1)* " " + seq2

  for k in range (len(seq1)):
    overlap_num=0
    #check individual characters to see if they match
    for i in range(len(string1)):
      overlap_num=0
      if string1[i]==string2[i]:
        for w in range(i, len(string1)):
            if string1[w]==string2[w]:
              overlap_num+=1
            else:
              overlap_num=0
              break
        if overlap_num == found_overlap:
          #count the number of left spaces in the second sequence
          spaces=0
          for j in range(len(string2)):
            if string2[j]==" ":
              spaces+=1
            else:
              break
          merged_String= string1[:spaces]+string2[spaces:]

          return merged_String.strip()
    string2= string2[1:]+" "

def no_merges(seq_matrix):
  for row in range(len(seq_matrix)):
    for col in range(len(seq_matrix)):
      if seq_matrix[row][col]>0:
        return False
  return True


# Function to perform greedy overlap-based assembly
def heuristic_assemble(short_reads):
  # keep looping until only one sequence is left
  for seq in range(len(short_reads)-1):
    seq_matrix = []
    for item in short_reads:
      seq_matrix.append(len(short_reads) * [0])

    for row in range(len(seq_matrix)):
      for col in range(len(seq_matrix)):
        # the diagonal is how much it overlaps with itself
        if row==col:
          seq_matrix[row][col] = -1
        else:
          seq_matrix[row][col] = find_high_overlap(short_reads[row], short_reads[col])


    if no_merges(seq_matrix)==True:
      return short_reads

    # find the highest overlap
    highest = [0]
    for i in range(len(seq_matrix)):
      for j in range(len(seq_matrix)):
        if seq_matrix[i][j]> highest[0]:
          highest[0]=seq_matrix[i][j]
          if len(highest)>1:
            highest[1]=i
            highest[2]=j
          else:
            highest.extend([i,j])

    seq1=short_reads[highest[1]]
    seq2=short_reads[highest[2]]

    short_reads.append(merge(seq1,seq2, highest[0]))
    short_reads.remove(seq1)
    short_reads.remove(seq2)

  return short_reads

# Perform greedy overlap-based assembly
assembled_sequence = heuristic_assemble(short_reads[:10])


### Your alignment score should be at least 1250

In [44]:
print(len(assembled_sequence[-1]))

3981


In [40]:
# Calculate the local alignment score using Smith-Waterman algorithm
alignment_score = smith_waterman(refrence_genome, assembled_sequence[-1])

# Print the local alignment score
print("Local Alignment Score:", alignment_score)

Local Alignment Score: 3748


In [ ]:

def perform_de_novo_assembly(short_reads, k):
    kmers = {}

    for read in short_reads:
      # added kmer of each read to dictionary
      for i in range(len(read) - k - 1):
          kmers[read[i : i+k]] = read[i+1 : i+k+1]

    kmer = list(kmers.keys())[0]
    assembled_sequence = kmer[:-1]  # added first kmer to assembled sequence

    while True:
        if kmer in kmers:
          # each step we add last character of kmer to assembled_sequence
          assembled_sequence += kmer[-1]
          kmer = kmers.pop(kmer)

        # if exact kmer was not in kmers dictionary
        else:
          best_diff = k + 1 # bigger than kmer size
          matched_kmer = ''
          finished = True

          # finding a kmer with least difference to match
          for seq in list(kmers.keys()):
            diff = 0
            for i in range(len(seq)):
              diff += 1 if kmer[i] != seq[i] else 0

            if best_diff > diff:
              best_diff = diff
              matched_kmer = seq
              # if we found a kmer with 5 differences at most we can continue to assembly
              if diff <= 5:
                finished = False
              else:
                finished = True

          if finished:
            return assembled_sequence
          else:
            finished = True
            assembled_sequence += matched_kmer[-1]
            kmer = kmers.pop(matched_kmer)


de_brujin_assembled_sequence = perform_de_novo_assembly(short_reads, k=25)


### Your alignment score should be at least 3800

In [26]:
len(de_brujin_assembled_sequence)

5414

In [27]:
alignment_score = smith_waterman(refrence_genome, de_brujin_assembled_sequence)

# Print the local alignment score
print("Local Alignment Score:", alignment_score)


Local Alignment Score: 8892
